### Conexão database

In [79]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

usuario = 'postgres'
senha = quote_plus('@manaus')
host = 'localhost'
porta = '5432'
nome_do_banco = 'db_mrp'

# Configuração da conexão com o PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{nome_do_banco}')


# Preparação de Dados para Banco de Dados

Este notebook demonstra o processo de preparação de uma planilha para inserção em um banco de dados. Os passos incluem:
- Carregar o arquivo Excel
- Limpar e ajustar os dados
- Tratar valores nulos
- Exportar os dados para um banco de dados SQL

## 1. Carregar o arquivo Excel

In [ ]:
import pandas as pd

# Carregar a planilha

file_path = r'C:\Users\Paulo\Documents\project01\data\Delivery Status 12042024 0833.xlsx'
xls = pd.ExcelFile(file_path)

# Carregar a planilha específica
df_delivery_status = xls.parse('Sheet1', header=0)

# Exibir as primeiras linhas
df_delivery_status.head()

,ORG,Item,UIT,Delivery Type,Supplier Code,Name,Departure No,Inspection Flag,PO,PO Remain,...,Item Cost,Po Currency Code,Po Price,WMS Item,Departure.2,Departure Cancel.2,Arrival.2,Arrival Cancel.2,Receiving.2,Nota No
0,NWK,MCK71451301,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000143,Y,6360,490,...,"4,69377240603417914894845871530644573707",BRL,"5,3186",N,LGESCS,NaN,SP917127,NaN,SP917866,179008
1,NWK,MCK71445802,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000142,N,1140,1140,...,"2,69644787424278107657597232303828107727",BRL,"3,0847",Y,LGESCS,LGESCS,NaN,NaN,NaN,179007
2,NWK,MCK71450401,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000144,N,8000,1700,...,"4,1280599385712045517693792343966154518",BRL,"4,5213",N,LGESCS,NaN,SP917127,NaN,SP917866,179007
3,NWK,ACQ30522223,R,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000145,Y,1086,0,...,"124,834848137643411952412334936010838004",BRL,"66,7765",N,LGESCS,NaN,AZ102331,NaN,SP917866,179012
4,NWK,ACQ30522223,R,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000145,Y,1086,0,...,"124,834848137643411952412334936010838004",BRL,"66,7765",N,LGESCS,NaN,AZ102331,NaN,SP917866,179012


In [81]:
df_delivery_status.columns

Index(['ORG', 'Item', 'UIT', 'Delivery Type', 'Supplier Code', 'Name',
       'Departure No', 'Inspection Flag', 'PO', 'PO Remain', 'Departure',
       'Departure Cancel', 'Arrival', 'Arrival Cancel', 'IQC Status',
       'Receiving', 'PO No', 'Kanban Code', 'Work Order', 'Line',
       'PO Subinventory', 'PO Locator', 'PO Creation', 'PO Due', 'Departure.1',
       'Departure Cancel.1', 'Arrival.1', 'Arrival Cancel.1', 'IQC Judgement',
       'Receiving.1', 'Planner', 'Uom', 'Purchaser', 'W-Keeper', 'Desc',
       'Spec', 'Item Cost', 'Po Currency Code', 'Po Price', 'WMS Item',
       'Departure.2', 'Departure Cancel.2', 'Arrival.2', 'Arrival Cancel.2',
       'Receiving.2', 'Nota No'],
      dtype='object')

### Converter coluna Receiving.1 para datetime

In [82]:
# Supondo que seu DataFrame seja 'df', converta a coluna 'Receiving.1' para datetime
df_delivery_status['Receiving.1'] = pd.to_datetime(df_delivery_status['Receiving.1'], errors='coerce')

# Verificando a conversão
print(df_delivery_status['Receiving.1'].head())


0   2024-11-27 16:37:30
1                   NaT
2   2024-11-27 12:57:11
3   2024-11-27 16:37:30
4   2024-11-27 16:37:30
Name: Receiving.1, dtype: datetime64[ns]


C:\Users\Paulo\AppData\Local\Temp\ipykernel_6660\3885989555.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_delivery_status['Receiving.1'] = pd.to_datetime(df_delivery_status['Receiving.1'], errors='coerce')


### Adicionar colunas "Status" e "Group"

In [83]:
df_colum_s_g = pd.DataFrame({
    'Status': [''] * len(df_delivery_status),  # Inicializando com valores vazios, ou você pode adicionar valores específicos
    'Group': [''] * len(df_delivery_status),   # Inicializando com valores vazios
})

df_delivery_status = pd.concat([df_colum_s_g, df_delivery_status], axis=1)

In [84]:
df_delivery_status

,Status,Group,ORG,Item,UIT,Delivery Type,Supplier Code,Name,Departure No,Inspection Flag,...,Item Cost,Po Currency Code,Po Price,WMS Item,Departure.2,Departure Cancel.2,Arrival.2,Arrival Cancel.2,Receiving.2,Nota No
0,,,NWK,MCK71451301,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000143,Y,...,"4,69377240603417914894845871530644573707",BRL,"5,3186",N,LGESCS,NaN,SP917127,NaN,SP917866,179008
1,,,NWK,MCK71445802,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000142,N,...,"2,69644787424278107657597232303828107727",BRL,"3,0847",Y,LGESCS,LGESCS,NaN,NaN,NaN,179007
2,,,NWK,MCK71450401,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000144,N,...,"4,1280599385712045517693792343966154518",BRL,"4,5213",N,LGESCS,NaN,SP917127,NaN,SP917866,179007
3,,,NWK,ACQ30522223,R,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000145,Y,...,"124,834848137643411952412334936010838004",BRL,"66,7765",N,LGESCS,NaN,AZ102331,NaN,SP917866,179012
4,,,NWK,ACQ30522223,R,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000145,Y,...,"124,834848137643411952412334936010838004",BRL,"66,7765",N,LGESCS,NaN,AZ102331,NaN,SP917866,179012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,,,NWH,AAN76409415,R,G,BR001203,ESSP_PAM INDUSTRIA DE PLASTICOS INJETADOS LTDA...,NWH24120000276,Y,...,"10,5914520520374986766764176500463592012",BRL,"6,7153",N,LGESCS,NaN,NaN,NaN,NaN,431017
2268,,,NWH,MAY68170801,G,G,BR013040,ESSP_KLABIN S.A._BR013040,NWH24120000277,N,...,"9,72779790473322662561114232334809520895",BRL,"10,46",N,LGESCS,NaN,NaN,NaN,NaN,233600
2269,,,NWH,MFZ67201001,G,G,BR001429,ESSP_KNAUF ISOPOR DA AMAZONIA LTDA._BR001429,NWH24120000278,N,...,"9,29070284344473254855413380221559427334",BRL,"9,99",N,LGESCS,NaN,NaN,NaN,NaN,201781
2270,,,NWH,MEZ66608901,G,G,BR001217,ESSP_LABEL PRESS IND. E COM. DA AMAZONIA LTDA_...,NWH24120000279,N,...,",189221765328561636832551829033925353937",BRL,",203",Y,LGESCS,NaN,NaN,NaN,NaN,282412


## Adição dos grupos

In [85]:
import pandas as pd

# Suponha que df seja seu DataFrame e a coluna 'Item' seja onde as condições se aplicam

# Definir as condições e os valores em uma lista de tuplas
groups = [
    (['NW1', 'NWD', 'NWH', 'NWW'], 'TV'),
    (['NW4', 'NWU', 'NWX'], 'AV'),
    (['NW7', 'NW8', 'NWQ'], 'AC'),
    (['NWK'], 'BM')
]

# Função para aplicar a lógica de acordo com as condições da lista
def calculate_group(item):
    for condition, group in groups:
        if item in condition:
            return group
    return 'Others'  # Caso não atenda a nenhuma das condições

# Aplicar a função para preencher a coluna 'Group'
df_delivery_status['Group'] = df_delivery_status['ORG'].apply(calculate_group)

# Exibindo o DataFrame resultante
df_delivery_status


,Status,Group,ORG,Item,UIT,Delivery Type,Supplier Code,Name,Departure No,Inspection Flag,...,Item Cost,Po Currency Code,Po Price,WMS Item,Departure.2,Departure Cancel.2,Arrival.2,Arrival Cancel.2,Receiving.2,Nota No
0,,BM,NWK,MCK71451301,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000143,Y,...,"4,69377240603417914894845871530644573707",BRL,"5,3186",N,LGESCS,NaN,SP917127,NaN,SP917866,179008
1,,BM,NWK,MCK71445802,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000142,N,...,"2,69644787424278107657597232303828107727",BRL,"3,0847",Y,LGESCS,LGESCS,NaN,NaN,NaN,179007
2,,BM,NWK,MCK71450401,G,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000144,N,...,"4,1280599385712045517693792343966154518",BRL,"4,5213",N,LGESCS,NaN,SP917127,NaN,SP917866,179007
3,,BM,NWK,ACQ30522223,R,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000145,Y,...,"124,834848137643411952412334936010838004",BRL,"66,7765",N,LGESCS,NaN,AZ102331,NaN,SP917866,179012
4,,BM,NWK,ACQ30522223,R,G,BR008197,ESSP_TSE INDUSTRIA DE ARTEFATOS ESTAMPADOS DE ...,NWK24110000145,Y,...,"124,834848137643411952412334936010838004",BRL,"66,7765",N,LGESCS,NaN,AZ102331,NaN,SP917866,179012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,,TV,NWH,AAN76409415,R,G,BR001203,ESSP_PAM INDUSTRIA DE PLASTICOS INJETADOS LTDA...,NWH24120000276,Y,...,"10,5914520520374986766764176500463592012",BRL,"6,7153",N,LGESCS,NaN,NaN,NaN,NaN,431017
2268,,TV,NWH,MAY68170801,G,G,BR013040,ESSP_KLABIN S.A._BR013040,NWH24120000277,N,...,"9,72779790473322662561114232334809520895",BRL,"10,46",N,LGESCS,NaN,NaN,NaN,NaN,233600
2269,,TV,NWH,MFZ67201001,G,G,BR001429,ESSP_KNAUF ISOPOR DA AMAZONIA LTDA._BR001429,NWH24120000278,N,...,"9,29070284344473254855413380221559427334",BRL,"9,99",N,LGESCS,NaN,NaN,NaN,NaN,201781
2270,,TV,NWH,MEZ66608901,G,G,BR001217,ESSP_LABEL PRESS IND. E COM. DA AMAZONIA LTDA_...,NWH24120000279,N,...,",189221765328561636832551829033925353937",BRL,",203",Y,LGESCS,NaN,NaN,NaN,NaN,282412


### Select da coluna date da table pph

In [86]:
# query = 'SELECT "Date" FROM table_pph;'
# df_dates_pph = pd.read_sql(query, engine)
# df_dates_pph


### Adicionar Y ou N na coluna Status

In [87]:
# # Pegando o valor da data de referência da tabela df_dates_pph
# #plan_pph_value = df_dates_pph['Date'].iloc[0]  # Pegando a primeira data ou ajuste conforme necessário
# # Definindo a data de referência fixada para 04/12/2024
# plan_pph_value = pd.to_datetime("2024-12-04 03:00:00")


# # Função para calcular o valor da coluna 'Status'
# def calculate_status(row):
#     # Verifica se 'Receiving.1' é vazio ou maior/igual ao valor de 'plan_pph_value'
#     if pd.isna(row['Receiving.1']) or row['Receiving.1'] >= plan_pph_value:
#         return 'Y'
#     else:
#         return 'N'

# # Adiciona a coluna 'Status' em df_delivery_status
# df_delivery_status['Status'] = df_delivery_status.apply(lambda row: calculate_status(row), axis=1)

# # Exibe o resultado
# print(df_delivery_status)


In [88]:
from datetime import datetime

# Passo 1: Definir a data de referência como a data atual
plan_pph_value = datetime.now()  # Pega a data atual como referência
print(f'plan_pph_value: {plan_pph_value}')

# Passo 2: Ajustar a consulta SQL
query = """
SELECT "Date" 
FROM table_pph 
WHERE "Date" >= %s
"""
print('query: {query}')
# Passo 3: Definir os parâmetros como uma lista de tuplas
params = (plan_pph_value, )

# Passo 4: Passar a consulta e o parâmetro corretamente para o pandas
df_reference_date = pd.read_sql(query, engine, params=params)
print(f'df_reference_date: {df_reference_date}')
# Passo 5: Se tiver alguma data correspondente, pega a mais recente
if not df_reference_date.empty:
    plan_pph_value = df_reference_date['Date'].max()  # Pegando a data mais recente

# Passo 6: Agora, você pode aplicar a lógica de "Status" baseado na data de referência dinâmica
# Função para calcular o valor da coluna 'Status' com base na data de referência dinâmica
def calculate_status(row):
    # Verifica se 'Receiving.1' é vazio ou maior/igual ao valor de 'plan_pph_value'
    if pd.isna(row['Receiving.1']) or row['Receiving.1'] >= plan_pph_value:
        return 'Y'
    else:
        return 'N'

# Passo 7: Adiciona a coluna 'Status' no DataFrame df_delivery_status
df_delivery_status['Status'] = df_delivery_status.apply(lambda row: calculate_status(row), axis=1)

# Exibe o resultado
print(df_delivery_status)


plan_pph_value: 2024-12-07 22:46:14.230323
query: {query}
df_reference_date:                      Date
0     2024-12-08 03:00:00
1     2024-12-08 03:00:00
2     2024-12-08 03:00:00
3     2024-12-08 03:00:00
4     2024-12-08 03:00:00
...                   ...
60139 2025-06-04 03:00:00
60140 2025-06-04 03:00:00
60141 2025-06-04 03:00:00
60142 2025-06-04 03:00:00
60143 2025-06-04 03:00:00

[60144 rows x 1 columns]
     Status Group  ORG         Item UIT Delivery Type Supplier Code  \
0         N    BM  NWK  MCK71451301   G             G      BR008197   
1         Y    BM  NWK  MCK71445802   G             G      BR008197   
2         N    BM  NWK  MCK71450401   G             G      BR008197   
3         N    BM  NWK  ACQ30522223   R             G      BR008197   
4         N    BM  NWK  ACQ30522223   R             G      BR008197   
...     ...   ...  ...          ...  ..           ...           ...   
2267      Y    TV  NWH  AAN76409415   R             G      BR001203   
2268      Y    TV

### Salvar no banco

In [89]:
# Exportar o DataFrame para o banco de dados PostgreSQL
df_delivery_status.to_sql('table_delivery_status', con=engine, if_exists='replace', index=False)
print('Dados salvos com sucesso no banco de dados PostgreSQL.')

Dados salvos com sucesso no banco de dados PostgreSQL.
